In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics .pairwise import sigmoid_kernel
import warnings
warnings.simplefilter('ignore')

movies = pd.read_csv("bollywood_full_1950-2019.csv")

movies_df= movies[{'original_title','story'}]
movies_df.drop_duplicates(subset='original_title',inplace=True)
movies_df.original_title = movies_df.original_title.str.lower()

movies_df['story'] = movies_df['story'].fillna('')

tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(movies_df['story'])

tfidf_matrix.shape

sig = sigmoid_kernel(tfidf_matrix,tfidf_matrix)

indices = pd.Series(movies_df.index, index = movies_df['original_title']).drop_duplicates()

In [9]:
def get_rec(title, sig=sig):
    try:
        title = title.lower()
        idx = indices[title]
        sim_score = list(enumerate(sig[idx]))
        sim_score = sorted(sim_score, key =lambda x:x[1],reverse=True)
        
        sim_score = sim_score[1:11]
        movie_indices = [i[0] for i in sim_score]
        return movies_df['original_title'].iloc[movie_indices]
    except:
        return "Movie Not available"

In [10]:
get_rec("raaz")

3694             jeevan mrityu
298     khel toh ab shuru hoga
2213          zakhmo ka hisaab
746                 united six
2915                   ashanti
4114                     daaka
3822        sadhu aur shaitaan
3190           dil se mile dil
3498               gehri chaal
2115             sangdil sanam
Name: original_title, dtype: object

In [18]:
import pickle

with open('sigmoid_kernel.pickle','wb') as f:
    pickle.dump(sig,f)

with open('indices.pickle','wb') as f:
    pickle.dump(indices,f)

with open('movie_title.pickle','wb') as f:
    pickle.dump(movies_df,f)